In [1]:
'''
Caleb Haynes 6/8/2020 for David Smith & Johanna Jarcho
Converts a set of behavioral text files in the AFNI format (seperated txt files) into one 'BIDS' standard file per participant
'''
import pandas as pd
import glob
import csv
from numpy import loadtxt

In [2]:
#make a list of all subject level directories to parse over
sub_file_list = glob.glob('*/*')

In [3]:
'''
main function to call in loop, converts a text file to a pandas dataframe with the BIDS format we want with columns for onset, 
duration and trial_type. We will want to join the dataframes per participant in the loop so we can sort by onset and maintain 
correct durations and task_types. 
''' 
#txt path string, duration int, trial_string
def fp_to_pd(txt_path, duration, trial_type):
        #I use glob.glob in the loop to find the file. This creates a list object so we use first item in list
        #([0]) and use numpy's loadtext module to grab the data inside the txt file. 
        txt_path = loadtxt(txt_path[0], delimiter=" ")
        #create an initial dictionary to load into pandas with our first column
        txt_dict = {'onset': txt_path}
        #instantiate pandas df with our dict
        df = pd.DataFrame(data = txt_dict)
        #add column for duration values
        df['duration'] = duration
        #add column with the trial_type
        df['trial_type'] = trial_type
        return df

In [49]:
for i in sub_file_list:
    #itertate over each subject, make a path string for each type of file, here decision, positive feedback, and negative 
    #feedback. 
    f_social_dec = glob.glob(i + '/social/decision/decisionPhase_social_*')
    f_social_fb_pos = glob.glob(i + '/social/feedback/pos*')
    f_social_fb_neg = glob.glob(i + '/social/feedback/neg*')
    
    f_doors_dec = glob.glob(i + '/doors/decision/decisionPhase_doors_*')
    f_doors_fb_pos = glob.glob(i + '/doors/feedback/pos*')
    f_doors_fb_neg = glob.glob(i + '/doors/feedback/neg*')
    
    #call function with each file path string, duration, and trial_type
    s_df_dec = fp_to_pd(f_social_dec, 3, 'decision')
    s_df_fb_pos = fp_to_pd(f_social_fb_pos, 1, 'win')
    s_df_fb_neg = fp_to_pd(f_social_fb_neg, 1, 'loss')
    
    d_df_dec = fp_to_pd(f_doors_dec, 3, 'decision')
    d_df_fb_pos = fp_to_pd(f_doors_fb_pos, 1, 'win')
    d_df_fb_neg = fp_to_pd(f_doors_fb_neg, 1, 'loss')
    
    #list of dataframes, merge together
    s_df_list = [s_df_dec,s_df_fb_pos,s_df_fb_neg]
    s_df_merge = pd.concat(s_df_list)
    
    d_df_list = [d_df_dec,d_df_fb_pos,d_df_fb_neg]
    d_df_merge = pd.concat(d_df_list)
    #sort rows by onset values and reset index
    df_soc = s_df_merge.sort_values(by=['onset']).reset_index()
    df_doors = d_df_merge.sort_values(by=['onset']).reset_index()
    
    soc_filename = f'sub-2{i[-3:]}_face-A.tsv'
    door_filename = f'sub-2{i[-3:]}_image-A.tsv'
    
    df_soc.to_csv(soc_filename, index=False, sep ='\t')
    df_doors.to_csv(door_filename, index=False, sep ='\t')